<a href="https://colab.research.google.com/github/baroodb/code/blob/main/flowers1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import os

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from pathlib import  Path

In [19]:
url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

In [20]:
data_path = tf.keras.utils.get_file(origin=url, fname='flower_photos', untar=True)
print(data_path)

228827136/228813984 [==============================] - 7s 0us/step
/root/.keras/datasets/flower_photos


In [21]:
path = Path(data_path)
path

PosixPath('/root/.keras/datasets/flower_photos')

In [22]:
entries = path.glob(pattern='*')
for entry in entries:
  if os.path.isfile(entry):
    Path(entry).unlink()

In [23]:
class_names = os.listdir(path)
class_names

['daisy', 'roses', 'dandelion', 'tulips', 'sunflowers']

In [24]:
list_ds = tf.data.Dataset.list_files(str(path/'*/*'), shuffle=False)
list_ds

<TensorSliceDataset shapes: (), types: tf.string>

In [25]:
def _get_labels(filename):
  parts = tf.strings.split(filename, os.path.sep)
  onehot = parts[-2] == class_names
  return tf.argmax(onehot)

def _process_image(filename):
  image = tf.io.read_file(filename)
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.convert_image_dtype(image, dtype=tf.float32)
  image = tf.image.resize(image, [224, 224])
  return image

def process_path(filename):
  label = _get_labels(filename)
  image = _process_image(filename)

  return image, label

def configure_for_perf(ds):
  ds = ds.cache()
  ds = ds.shuffle(buffer_size=5000)
  ds = ds.batch(32)
  ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)

  return ds

def get_datasets(path, val_size=400):
  filenames = tf.data.Dataset.list_files(file_pattern=str(path/'*/*'), shuffle=False)
  filenames = filenames.shuffle(buffer_size=5000, reshuffle_each_iteration=False)
  train_files = filenames.skip(val_size)
  val_files = filenames.take(val_size)
  return train_files, val_files

In [26]:
# build my  data pipeline 
train_files, val_files = get_datasets(path)

train_ds = train_files.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_files.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)

train_ds = configure_for_perf(train_ds)
val_ds = configure_for_perf(val_ds)

In [27]:
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Dense, Dropout, Flatten
from tensorflow.keras.models import Model
import tensorflow as tf
input_shape = (224, 224, 3)
data_augmentation = tf.keras.Sequential(
  [
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=input_shape),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

def build_model(input_shape):
  inputs = Input(input_shape)
  x = Conv2D(128, (3, 3), activation='relu')(inputs)
  x = MaxPool2D()(x)
  x = Dropout(0.2)(x)

  x = Conv2D(128, (3, 3), activation='relu')(x)
  x = MaxPool2D()(x)
  x = Dropout(0.2)(x)

  x = Conv2D(128, (3, 3), activation='relu')(x)
  x = MaxPool2D()(x)
  x = Dropout(0.2)(x)
  
  x = Flatten()(x)
  x = Dense(64, activation='relu')(x)
  
  outputs = Dense(5)(x)

  model = Model(inputs=inputs, outputs=outputs)

  model.compile(optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'], 
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True))

  return model

In [30]:
from tensorflow.keras import Sequential, layers
model = Sequential([
  data_augmentation,
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(5)
])

model.compile(optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'], 
              loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True))


In [31]:
#model = build_model((224, 224, 3))

H = model.fit(train_ds, 
              validation_data=val_ds,
              epochs=10)

Epoch 1/10
103/103 [==============================] - 43s 81ms/step - loss: 1.2553 - accuracy: 0.4682 - val_loss: 1.0816 - val_accuracy: 0.5900
Epoch 2/10
103/103 [==============================] - 7s 68ms/step - loss: 0.9814 - accuracy: 0.6174 - val_loss: 1.0024 - val_accuracy: 0.6175
Epoch 3/10
103/103 [==============================] - 7s 67ms/step - loss: 0.8820 - accuracy: 0.6572 - val_loss: 0.7978 - val_accuracy: 0.7025
Epoch 4/10
103/103 [==============================] - 7s 68ms/step - loss: 0.8115 - accuracy: 0.6905 - val_loss: 0.7532 - val_accuracy: 0.7150
Epoch 5/10
103/103 [==============================] - 7s 67ms/step - loss: 0.7873 - accuracy: 0.7076 - val_loss: 0.8331 - val_accuracy: 0.7150
Epoch 6/10
103/103 [==============================] - 7s 67ms/step - loss: 0.7092 - accuracy: 0.7303 - val_loss: 0.7272 - val_accuracy: 0.7175
Epoch 7/10
103/103 [==============================] - 7s 68ms/step - loss: 0.7059 - accuracy: 0.7306 - val_loss: 0.7083 - val_accuracy: 0.735

In [ ]:
tf.config.list_physical_devices()

In [ ]:
x = tf.random.uniform((3, 3))
x.device.endswith('GPU:0')